# 1. 倒水问题
- 9升的瓶子和4升的瓶子要得到6升的水

In [79]:
def pour_problem(X,Y,goal,start=(0,0)):
    """X and Y are the capacity of glasses;(x,y) is current fill levels and
    represents a state, The goal is a level that can be in either glass.
    Start at start state and follow successors until we reach the goal.
    keep track of frontier and previsouly explored;fail when no frontier
    从（0，0）开始进行所有可能路径的搜索，将可能的路径存储到frontier里面，
    将已经走过的状态存储到explored里面，避免走重复的路径
    """
    if goal in start:
        return [start]
    explored = set() # set of states we have visited
    frontier =[[start]] # ordered list of paths we have blazed
    while frontier:
        #print("++"*10)
        path = frontier.pop(0) # 移除第一个元素
        #print('path=',path)
        #print('f=',frontier)
        (x,y) = path[-1] #Last state in the first path of the frontier
        for (state,action) in successors(x,y,X,Y).items():
            if state not in explored:
                explored.add(state)
                path2 = path+[action,state]
                if goal in state:
                    return path2
                else:
                    frontier.append(path2)
                    #print('fl=',frontier)
        #print('e=',explored)
        #print("=="*10)
    return Fail
Fail=[]

def successors(x,y,X,Y):
    """return a dict of {state:action} pairs descrbing what can be reached 
    from the (x,y) state ,and how
    """
    assert(x<=X) and y<=Y ## (x,y) is glass levels; X and Y are glass sizes
    return {((0,y+x) if y+x<=Y else (x-(Y-y),y+(Y-y))):'X->Y',
            ((x+y,0) if x+y<=X else (x+(X-x),y-(X-x))):'X->Y',
            (X,y):'fill X',(x,Y):'fill Y',
            (0,y):'empty X',(x,0):'empty Y'}
print(pour_problem(4,9,7))

[(0, 0), 'fill X', (4, 0), 'X->Y', (0, 4), 'fill X', (4, 4), 'X->Y', (0, 8), 'fill X', (4, 8), 'X->Y', (3, 9), 'empty Y', (3, 0), 'X->Y', (0, 3), 'fill X', (4, 3), 'X->Y', (0, 7)]


In [2]:
# import doctest
# class Test:
#     """
#     >>> successors(0,0,4,9)
#     {(0,9):'fill Y',(0,0):'empty Y',(4,0):'fill X'}
#     >>> pour_problem(4,9,6)
#     """
# print(doctest.testmod())

# 2 brige problem
- 4个人过桥和一个手电筒，每个人分别需要1，2，5，10分钟
- 每次只能过两个人，且必须带手电筒
- 求最短的过桥时间

In [3]:
def bsuccessors(state):
    """Return a dict of {state:action} pairs. A state is a (here, there, t) tuple,
    where here and there are frozensets of people (indicated by their times) and/or
    the 'light', and t is a number indicating the elapsed time. Action is represented
    as a tuple (person1, person2, arrow), where arrow is '->' for here to there and 
    '<-' for there to here."""
    here, there, t = state
    if 'light' in here:
        return dict(((here - frozenset([a,b,'light']),
                    there | frozenset([a,b,'light']),
                      t+max(a,b)),(a,b,'->'))
                    for a in here if a is not 'light'
                    for b in here if b is not 'light'
                    )
    else:
        return dict(((here | frozenset([a,b,'light']),
                     there - frozenset([a,b,'light']),
                     t + max(a,b)),(a,b,'<-'))
                   for a in there if a is not 'light'
                   for b in there if b is not 'light')

In [4]:
def test():
    assert bsuccessors((frozenset([1, 'light']), frozenset([]), 3)) == {
                (frozenset([]), frozenset([1, 'light']), 4): (1, 1, '->')}

    assert bsuccessors((frozenset([]), frozenset([2, 'light']), 0)) =={
                (frozenset([2, 'light']), frozenset([]), 2): (2, 2, '<-')}
    
    return 'tests pass'

print(test())

tests pass


In [5]:
def bridge_problem(here):
    here = frozenset(here)|frozenset(['light'])
    explored = set()
    frontier = [[(here,frozenset(),0)]]
    if not here:
        return frontier[0]
    while frontier:
        path = frontier.pop(0)
        for (state,action) in bsuccessors(path[-1]).items():
            if state not in explored:
                here,there,t=state
                explored.add(state)
                path2 = path+[action,state]
                if not here: #here 为空，全部过桥
                    return path2
                else:
                    frontier.append(path2)
                    frontier.sort(key=elapsed_time)# 优先处理耗时短的
    return []
def elapsed_time(path):
    return path[-1][2]

In [6]:
print(bridge_problem([1,2,5,10]))

[(frozenset({1, 2, 5, 10, 'light'}), frozenset(), 0), (2, 1, '->'), (frozenset({10, 5}), frozenset({1, 2, 'light'}), 2), (1, 1, '<-'), (frozenset({1, 10, 5, 'light'}), frozenset({2}), 3), (5, 1, '->'), (frozenset({10}), frozenset({1, 2, 5, 'light'}), 8), (1, 1, '<-'), (frozenset({1, 10, 'light'}), frozenset({2, 5}), 9), (10, 1, '->'), (frozenset(), frozenset({1, 2, 5, 10, 'light'}), 19)]


这个答案是错误的，原因在于找到答案就停止了，没有继续寻找其他可能耗时最短的路径
![](https://tva1.sinaimg.cn/large/006y8mN6gy1g7g4mo6au0j31e40tqx26.jpg)

In [10]:
# 正确答案
def bsuccessors(state):
    """Return a dict of {state:action} pairs.  A state is a (here, there, t) tuple,
    where here and there are frozensets of people (indicated by their times) and/or
    the light, and t is a number indicating the elapsed time."""
    here, there, t = state
    if 'light' in here:
        return dict(((here  - frozenset([a,b, 'light']),
                      there | frozenset([a, b, 'light']),
                      t + max(a, b)),
                     (a, b, '->'))
                    for a in here if a is not 'light'
                    for b in here if b is not 'light')
    else:
        return dict(((here  | frozenset([a,b, 'light']),
                      there - frozenset([a, b, 'light']),
                      t + max(a, b)),
                     (a, b, '<-'))
                    for a in there if a is not 'light'
                    for b in there if b is not 'light')  
        
def elapsed_time(path):
    return path[-1][2]

def bridge_problem(here):
    """Modify this to test for goal later: after pulling a state off frontier,
    not when we are about to put it on the frontier."""
    ## modify code below
    here = frozenset(here) | frozenset(['light'])
    explored = set() # set of states we have visited
    # State will be a (people-here, people-there, time-elapsed)
    frontier = [ [(here, frozenset(), 0)] ] # ordered list of paths we have blazed
    if not here:
        return frontier[0]
    while frontier:
        path = frontier.pop(0)
        here1,there1,t1 = path[-1]
        if not here1 or here1 == set(['light']):
            return path
        for (state, action) in bsuccessors(path[-1]).items():
            if state not in explored:
                here, there, t = state
                explored.add(state)
                path2 = path + [action, state]
                # 此处即使找到答案也不能停止，需要将时间短的继续搜索
                frontier.append(path2)
                frontier.sort(key=elapsed_time)
    return []

def test():
    assert bridge_problem(frozenset((1, 2),))[-1][-1] == 2 # the [-1][-1] grabs the total elapsed time
    assert bridge_problem(frozenset((1, 2, 5, 10),))[-1][-1] == 17
    return 'tests pass'

print(test())

print(bridge_problem([1,2,5,10]))

tests pass
[(frozenset({1, 2, 5, 10, 'light'}), frozenset(), 0), (2, 1, '->'), (frozenset({10, 5}), frozenset({1, 2, 'light'}), 2), (1, 1, '<-'), (frozenset({1, 10, 5, 'light'}), frozenset({2}), 3), (5, 10, '->'), (frozenset({1}), frozenset({2, 10, 5, 'light'}), 13), (2, 2, '<-'), (frozenset({1, 2, 'light'}), frozenset({10, 5}), 15), (2, 1, '->'), (frozenset(), frozenset({1, 2, 5, 10, 'light'}), 17)]


In [11]:
def path_states(path):
    "Return a list of states in this path."
    return path[0::2]
    
def path_actions(path):
    "Return a list of actions in this path."
    return path[1::2]

import doctest
class TestBridge: """
>>> elapsed_time(bridge_problem([1,2,5,10]))
17

## There are two equally good solutions
>>> S1 = [(2, 1, '->'), (1, 1, '<-'), (5, 10, '->'), (2, 2, '<-'), (2, 1, '->')]
>>> S2 = [(2, 1, '->'), (2, 2, '<-'), (5, 10, '->'), (1, 1, '<-'), (2, 1, '->')]
>>> path_actions(bridge_problem([1,2,5,10])) in (S1, S2)
True

## Try some other problems
>>> path_actions(bridge_problem([1,2,5,10,15,20]))
[(2, 1, '->'), (1, 1, '<-'), (10, 5, '->'), (2, 2, '<-'), (2, 1, '->'), (1, 1, '<-'), (15, 20, '->'), (2, 2, '<-'), (2, 1, '->')]

>>> path_actions(bridge_problem([1,2,4,8,16,32]))
[(2, 1, '->'), (1, 1, '<-'), (8, 4, '->'), (2, 2, '<-'), (1, 2, '->'), (1, 1, '<-'), (16, 32, '->'), (2, 2, '<-'), (2, 1, '->')]

>>> [elapsed_time(bridge_problem([1,2,4,8,16][:N])) for N in range(6)]
[0, 1, 2, 7, 15, 28]

>>> [elapsed_time(bridge_problem([1,1,2,3,5,8,13,21][:N])) for N in range(8)]
[0, 1, 1, 2, 6, 12, 19, 30]

"""

print(doctest.testmod())

**********************************************************************
File "__main__", line ?, in __main__.TestBridge
Failed example:
    path_actions(bridge_problem([1,2,4,8,16,32]))
Expected:
    [(2, 1, '->'), (1, 1, '<-'), (8, 4, '->'), (2, 2, '<-'), (1, 2, '->'), (1, 1, '<-'), (16, 32, '->'), (2, 2, '<-'), (2, 1, '->')]
Got:
    [(2, 1, '->'), (1, 1, '<-'), (8, 4, '->'), (2, 2, '<-'), (2, 1, '->'), (1, 1, '<-'), (16, 32, '->'), (2, 2, '<-'), (2, 1, '->')]
**********************************************************************
1 items had failures:
   1 of   8 in __main__.TestBridge
***Test Failed*** 1 failures.
TestResults(failed=1, attempted=8)


## 2.1 优化效率
- 存在状态一样，但是耗时不一样的情况

In [17]:
# -----------------
# User Instructions
# 
# write a function, bsuccessors2 that takes a state as input
# and returns a dictionary of {state:action} pairs.
# 
# The new representation for a path should be a list of 
# [state, (action, total time), state, ... , ], though this 
# function will just return {state:action} pairs and will
# ignore total time. 
#
# The previous bsuccessors function is included for your reference.

def bsuccessors2(state):
    """Return a dict of {state:action} pairs. A state is a
    (here, there) tuple, where here and there are frozensets
    of people (indicated by their travel times) and/or the light."""
    # your code here
    here, there = state
    if 'light' in here:
        return dict(((here  - frozenset([a,b, 'light']),
                      there | frozenset([a, b, 'light']),
                      ),
                     (a, b, '->'))
                    for a in here if a is not 'light'
                    for b in here if b is not 'light')
    else:
        return dict(((here  | frozenset([a,b, 'light']),
                      there - frozenset([a, b, 'light']),
                     ),
                     (a, b, '<-'))
                    for a in there if a is not 'light'
                    for b in there if b is not 'light')  

def bsuccessors(state):
    """Return a dict of {state:action} pairs.  A state is a (here, there, t) tuple,
    where here and there are frozensets of people (indicated by their times) and/or
    the light, and t is a number indicating the elapsed time."""
    here, there, t = state
    if 'light' in here:
        return dict(((here  - frozenset([a,b, 'light']),
                      there | frozenset([a, b, 'light']),
                      t + max(a, b)),
                     (a, b, '->'))
                    for a in here if a is not 'light'
                    for b in here if b is not 'light')
    else:
        return dict(((here  | frozenset([a,b, 'light']),
                      there - frozenset([a, b, 'light']),
                      t + max(a, b)),
                     (a, b, '<-'))
                    for a in there if a is not 'light'
                    for b in there if b is not 'light')  


def test():
    here1 = frozenset([1, 'light']) 
    there1 = frozenset([])

    here2 = frozenset([1, 2, 'light'])
    there2 = frozenset([3])
    
    assert bsuccessors2((here1, there1)) == {
            (frozenset([]), frozenset([1, 'light'])): (1, 1, '->')}
    assert bsuccessors2((here2, there2)) == {
            (frozenset([1]), frozenset(['light', 2, 3])): (2, 2, '->'), 
            (frozenset([2]), frozenset([1, 3, 'light'])): (1, 1, '->'), 
            (frozenset([]), frozenset([1, 2, 3, 'light'])): (2, 1, '->')}
    return 'tests pass'
print(test())

here2 = frozenset([1, 2, 'light'])
there2 = frozenset([3])
print(bsuccessors2((here2, there2)))

tests pass
{(frozenset({2}), frozenset({1, 3, 'light'})): (1, 1, '->'), (frozenset(), frozenset({1, 2, 3, 'light'})): (2, 1, '->'), (frozenset({1}), frozenset({2, 3, 'light'})): (2, 2, '->')}


In [21]:
def path_cost(path):
    """The total cost of a path (which is stored in a tuple
    with the final action."""
    # path = (state, (action, total_cost), state, ... )
    if len(path) < 3:
        return 0
    else:
        return path[-2][-1]
        
def bcost(action):
    """Returns the cost (a number) of an action in the
    bridge problem."""
    # An action is an (a, b, arrow) tuple; a and b are 
    # times; arrow is a string. 
    a, b, arrow = action
    return max(a,b)

def bridge_problem2(here):
    here = frozenset(here) | frozenset(['light'])
    explored = set() # set of states we have visited
    # State will be a (people-here, people-there, time-elapsed)
    frontier = [ [(here, frozenset())] ] # ordered list of paths we have blazed

    while frontier:
        path = frontier.pop(0)
        here1,there1=state1= path[-1]
        if not here1 or (len(here1)==1 and 'light' in here1):
            return path
        explored.add(state1)
        pcost = path_cost(path)
        for (state, action) in bsuccessors2(state1).items():
            if state not in explored:
                total_cost = pcost+bcost(action)
                path2 = path + [(action,total_cost), state]
                add_to_frontier(frontier,path2)
    return []

def add_to_frontier(frontier,path):
    "add path to frontier,replacing costlier path is there is one"
    old =None
    for i,p in enumerate(frontier):
        if p[-1] == path[-1]:
            old=i
            break
    if old is not None and path_cost(frontier[old])<path_cost(path):
        return #old path was better
    elif old is not None:
        del frontier[old] #old path was worse
    frontier.append(path)

In [22]:
print(bridge_problem2(frozenset((1, 2, 5, 10),)))

[(frozenset({1, 2, 5, 10, 'light'}), frozenset()), ((2, 1, '->'), 2), (frozenset({10, 5}), frozenset({1, 2, 'light'})), ((2, 2, '<-'), 4), (frozenset({10, 2, 5, 'light'}), frozenset({1})), ((5, 10, '->'), 14), (frozenset({2}), frozenset({1, 10, 5, 'light'})), ((1, 1, '<-'), 15), (frozenset({1, 2, 'light'}), frozenset({10, 5})), ((2, 1, '->'), 17), (frozenset(), frozenset({1, 2, 5, 10, 'light'}))]


# 3 传教士和野人问题
- 3个传教士和3个野人过河，每次只能过2人，两边传教士人数要大于野人

In [74]:
# -----------------
# User Instructions
# 
# Write a function, csuccessors, that takes a state (as defined below) 
# as input and returns a dictionary of {state:action} pairs. 
#
# A state is a tuple with six entries: (M1, C1, B1, M2, C2, B2), where 
# M1 means 'number of missionaries on the left side.'
#
# An action is one of the following ten strings: 
#
# 'MM->', 'MC->', 'CC->', 'M->', 'C->', '<-MM', '<-MC', '<-M', '<-C', '<-CC'
# where 'MM->' means two missionaries travel to the right side.
# 
# We should generate successor states that include more cannibals than
# missionaries, but such a state should generate no successors.

def csuccessors(state):
    """Find successors (including those that result in dining) to this
    state. But a state where the cannibals can dine has no successors."""
    M1, C1, B1, M2, C2, B2 = state
    if C1>M1>0 or C2>M2>0:
        return {}
    item=[]
    if B1==1:
        item += [(sub(state,d),a+'->') for d,a in delta.items()]
    elif B2 == 1:
        item += [(add(state,d),'<-'+a) for d,a in delta.items()]
    # 剔除<0 
    item = filter(lambda x:False not in [j>=0 for j in x[0]] ,item)
        
    return dict(item)

delta = {(2, 0, 1,    -2, 0, -1): 'MM', 
        (0, 2, 1,     0, -2, -1): 'CC', 
        (1, 1, 1,     -1, -1, -1): 'MC', 
        (1, 0, 1,     -1, 0, -1): 'M', 
        (0, 1, 1,      0, -1, -1): 'C'}
        
def add(X,Y):
    "and two vetors X and Y"
    return tuple(x+y for x,y in zip(X,Y))
def sub(X,Y):
    "substrac vector Y from X"
    return tuple(x-y for x,y in zip(X,Y))

def test():
    assert csuccessors((2, 2, 1, 0, 0, 0)) == {(2, 1, 0, 0, 1, 1): 'C->', 
                                               (1, 2, 0, 1, 0, 1): 'M->', 
                                               (0, 2, 0, 2, 0, 1): 'MM->', 
                                               (1, 1, 0, 1, 1, 1): 'MC->', 
                                               (2, 0, 0, 0, 2, 1): 'CC->'}
    assert csuccessors((1, 1, 0, 4, 3, 1)) == {(1, 2, 1, 4, 2, 0): '<-C', 
                                               (2, 1, 1, 3, 3, 0): '<-M', 
                                               (3, 1, 1, 2, 3, 0): '<-MM', 
                                               (1, 3, 1, 4, 1, 0): '<-CC', 
                                               (2, 2, 1, 3, 2, 0): '<-MC'}
    assert csuccessors((1, 4, 1, 2, 2, 0)) == {}
    return 'tests pass'

#print(test())
print(csuccessors((2, 0, 1, 0, 0, 0)))

{(0, 0, 0, 2, 0, 1): 'MM->', (1, 0, 0, 1, 0, 1): 'M->'}


In [75]:
def mc_problem(start=(3,3,1,0,0,0),goal=None):
    """
    """
    if goal is None:
        goal = (0,0,0)+start[:3]
    if start == goal:
        return start
    explored = set()
    frontier = [[start]]
    while frontier:
        path = frontier.pop(0)
        s = path[-1]
        for state,action in csuccessors(s).items():
            if state not in explored:
                explored.add(state)
                path2 = path + [action,state]
                if state == goal:
                    return path2
                else:
                    frontier.append(path2)
    return []

In [76]:
print(mc_problem())

[(3, 3, 1, 0, 0, 0), 'CC->', (3, 1, 0, 0, 2, 1), '<-C', (3, 2, 1, 0, 1, 0), 'CC->', (3, 0, 0, 0, 3, 1), '<-C', (3, 1, 1, 0, 2, 0), 'MM->', (1, 1, 0, 2, 2, 1), '<-MC', (2, 2, 1, 1, 1, 0), 'MM->', (0, 2, 0, 3, 1, 1), '<-C', (0, 3, 1, 3, 0, 0), 'CC->', (0, 1, 0, 3, 2, 1), '<-M', (1, 1, 1, 2, 2, 0), 'MC->', (0, 0, 0, 3, 3, 1)]


In [84]:
def mc_problem2(start=(3, 3, 1, 0, 0, 0), goal=None):
    # your code here if necessary
    if goal is None:
        goal = (0,0,0)+start[:3]
    return shortest_path_search(start,csuccessors,all_gone) # <== insert arguments here
def all_gone(state):return state[:3]==(0,0,0)
def shortest_path_search(start, successors, is_goal):
    """Find the shortest path from start state to a state
    such that is_goal(state) is true."""
    if is_goal(start):
        return [start]
    explored = set()
    frontier = [ [start] ] 
    while frontier:
        path = frontier.pop(0)
        s = path[-1]
        for (state, action) in successors(s).items():
            if state not in explored:
                explored.add(state)
                path2 = path + [action, state]
                if is_goal(state):
                    return path2
                else:
                    frontier.append(path2)
    return Fail
Fail = []


    

In [86]:
# assert mc_problem2() == [(3, 3, 1, 0, 0, 0), 'CC->', 
#                              (3, 1, 0, 0, 2, 1), '<-C', 
#                              (3, 2, 1, 0, 1, 0), 'CC->', 
#                              (3, 0, 0, 0, 3, 1), '<-C', 
#                              (3, 1, 1, 0, 2, 0), 'MM->', 
#                              (1, 1, 0, 2, 2, 1), '<-MC', 
#                              (2, 2, 1, 1, 1, 0), 'MM->', 
#                              (0, 2, 0, 3, 1, 1), '<-C', 
#                              (0, 3, 1, 3, 0, 0), 'CC->', 
#                              (0, 1, 0, 3, 2, 1), '<-C', 
#                              (0, 2, 1, 3, 1, 0), 'CC->', 
#                              (0, 0, 0, 3, 3, 1)]
print(mc_problem2())


[(3, 3, 1, 0, 0, 0), 'CC->', (3, 1, 0, 0, 2, 1), '<-C', (3, 2, 1, 0, 1, 0), 'CC->', (3, 0, 0, 0, 3, 1), '<-C', (3, 1, 1, 0, 2, 0), 'MM->', (1, 1, 0, 2, 2, 1), '<-MC', (2, 2, 1, 1, 1, 0), 'MM->', (0, 2, 0, 3, 1, 1), '<-C', (0, 3, 1, 3, 0, 0), 'CC->', (0, 1, 0, 3, 2, 1), '<-M', (1, 1, 1, 2, 2, 0), 'MC->', (0, 0, 0, 3, 3, 1)]


# 4 作业

## 4.1 重构bsuccessor

In [101]:
# -----------------
# User Instructions
# 
# In this problem you will be refactoring the bsuccessors function.
# Your new function, bsuccessors3, will take a state as an input
# and return a dict of {state:action} pairs. 
#
# A state is a (here, there, light) tuple. Here and there are 
# frozensets of people (each person is represented by an integer
# which corresponds to their travel time), and light is 0 if 
# it is on the `here` side and 1 if it is on the `there` side.
#
# An action is a tuple of (travelers, arrow), where the arrow is
# '->' or '<-'. See the test() function below for some examples
# of what your function's input and output should look like.

def bsuccessors3(state):
    """Return a dict of {state:action} pairs.  State is (here, there, light)
    where here and there are frozen sets of people, light is 0 if the light is 
    on the here side and 1 if it is on the there side.
    Action is a tuple (travelers, arrow) where arrow is '->' or '<-'"""
    here,there,light = state
    if light == 0: #light is here
        return dict(((here  - frozenset([a,b]),
                      there | frozenset([a, b]),
                      light+1),
                     (set([a,b]), '->'))
                    for a in here for b in here )
    else:
        return dict(((here  | frozenset([a,b]),
                      there - frozenset([a, b]),
                      light-1),
                     (set([a,b]), '<-'))
                    for a in there for b in there )  

def test():
    print(bsuccessors3((frozenset([1]), frozenset([]), 0)))
    assert bsuccessors3((frozenset([1]), frozenset([]), 0)) == {
            (frozenset([]), frozenset([1]), 1)  :  (set([1]), '->')}

    assert bsuccessors3((frozenset([1, 2]), frozenset([]), 0)) == {
            (frozenset([1]), frozenset([2]), 1)    :  (set([2]), '->'), 
            (frozenset([]), frozenset([1, 2]), 1)  :  (set([1, 2]), '->'), 
            (frozenset([2]), frozenset([1]), 1)    :  (set([1]), '->')}

    assert bsuccessors3((frozenset([2, 4]), frozenset([3, 5]), 1)) == {
            (frozenset([2, 4, 5]), frozenset([3]), 0)   :  (set([5]), '<-'), 
            (frozenset([2, 3, 4, 5]), frozenset([]), 0) :  (set([3, 5]), '<-'), 
            (frozenset([2, 3, 4]), frozenset([5]), 0)   :  (set([3]), '<-')}
    return 'tests pass'

print(test())


{(frozenset(), frozenset({1}), 1): ({1}, '->')}
tests pass


## 4.2 多个杯子的问题

In [105]:
# -----------------
# User Instructions
# 
# In this problem, you will solve the pouring problem for an arbitrary
# number of glasses. Write a function, more_pour_problem, that takes 
# as input capacities, goal, and (optionally) start. This function should 
# return a path of states and actions.
#
# Capacities is a tuple of numbers, where each number represents the 
# volume of a glass. 
#
# Goal is the desired volume and start is a tuple of the starting levels
# in each glass. Start defaults to None (all glasses empty).
#
# The returned path should look like [state, action, state, action, ... ]
# where state is a tuple of volumes and action is one of ('fill', i), 
# ('empty', i), ('pour', i, j) where i and j are indices indicating the 
# glass number. 



def more_pour_problem(capacities, goal, start=None):
    """The first argument is a tuple of capacities (numbers) of glasses; the
    goal is a number which we must achieve in some glass.  start is a tuple
    of starting levels for each glass; if None, that means 0 for all.
    Start at start state and follow successors until we reach the goal.
    Keep track of frontier and previously explored; fail when no frontier.
    On success return a path: a [state, action, state2, ...] list, where an
    action is one of ('fill', i), ('empty', i), ('pour', i, j), where
    i and j are indices indicating the glass number."""
    # your code here
    def is_goal(state):
        return goal in state
    def more_successors(state):
        succ = {}
        for i in range(len(state)):
            succ[replace(state,i,capacities[i])] = ('fill',i)
            succ[replace(state,i,0)] = ('empty',i)
            for j in range(len(state)):
                if i!=j:
                    # 将i中的水倒入j中
                    amount = min(state[i],capacities[j]-state[j]) #i中已有的水和j中剩余的水，取小的
                    state2 = replace(state,i,state[i]-amount)
                    succ[replace(state2,j,state[j]+amount)]=('pour',i,j)
        return suc
    if start is None:start = (0,)*len(capacities)
    return shortest_path_search(start,more_successors,is_goal)

def replace(seq,i,val):
    s=list(seq)
    s[i]=val
    return type(seq)(s)

def shortest_path_search(start, successors, is_goal):
    """Find the shortest path from start state to a state
    such that is_goal(state) is true."""
    if is_goal(start):
        return [start]
    explored = set()
    frontier = [ [start] ] 
    while frontier:
        path = frontier.pop(0)
        s = path[-1]
        for (state, action) in successors(s).items():
            if state not in explored:
                explored.add(state)
                path2 = path + [action, state]
                if is_goal(state):
                    return path2
                else:
                    frontier.append(path2)
    return Fail

Fail = []
    
def test_more_pour():
    assert more_pour_problem((1, 2, 4, 8), 4) == [
        (0, 0, 0, 0), ('fill', 2), (0, 0, 4, 0)]
    assert more_pour_problem((1, 2, 4), 3) == [
        (0, 0, 0), ('fill', 2), (0, 0, 4), ('pour', 2, 0), (1, 0, 3)] 
    starbucks = (8, 12, 16, 20, 24)
    assert not any(more_pour_problem(starbucks, odd) for odd in (3, 5, 7, 9))
    assert all(more_pour_problem((1, 3, 9, 27), n) for n in range(28))
    assert more_pour_problem((1, 3, 9, 27), 28) == []
    return 'test_more_pour passes'

print(test_more_pour())

test_more_pour passes


## 4.3 地铁规划

In [169]:
# -----------------
# User Instructions
# 
# Write a function, subway, that takes lines as input (read more about
# the **lines notation in the instructor comments box below) and returns
# a dictionary of the form {station:{neighbor:line, ...}, ... } 
#
# For example, when calling subway(boston), one of the entries in the 
# resulting dictionary should be 'foresthills': {'backbay': 'orange'}. 
# This means that foresthills only has one neighbor ('backbay') and 
# that neighbor is on the orange line. Other stations have more neighbors:
# 'state', for example, has 4 neighbors.
#
# Once you've defined your subway function, you can define a ride and 
# longest_ride function. ride(here, there, system) takes as input 
# a starting station (here), a destination station (there), and a subway
# system and returns the shortest path.
#
# longest_ride(system) returns the longest possible ride in a given 
# subway system. 

# -------------
# Grading Notes
#
# The subway() function will not be tested directly, only ride() and 
# longest_ride() will be explicitly tested. If your code passes the 
# assert statements in test_ride(), it should be marked correct.
import collections
def subway(**lines):
    """Define a subway map. Input is subway(linename='station1 station2...'...).
    Convert that and return a dict of the form: {station:{neighbor:line,...},...}"""
    ## your code here
#     res={}
#     stations = set([s for ll,ss in lines.items() for s in ss.split(' ')])
#     for s in stations:
#         nbs = {}
#         for k,v in lines.items():
#             vlist = v.split(' ')
#             try:
#                 index = vlist.index(s)
#                 if index != 0:
#                     nbs[vlist[index-1]]=k
#                 if index != len(vlist)-1:
#                     nbs[vlist[index+1]]=k
#             except:
#                 pass
#         res[s]=nbs
#     return res
    successors = collections.defaultdict(dict)
    for linename,stops in lines.items():
        for a,b in overlapping_pairs(stops.split()):
            successors[a][b]=linename
            successors[b][a]=linename
    return successors
def overlapping_pairs(items):
    return [items[i:i+2] for i in range(len(items)-1)]
        

boston = subway(
    blue='bowdoin government state aquarium maverick airport suffolk revere wonderland',
    orange='oakgrove sullivan haymarket state downtown chinatown tufts backbay foresthills',
    green='lechmere science north haymarket government park copley kenmore newton riverside',
    red='alewife davis porter harvard central mit charles park downtown south umass mattapan')
#print(boston)    
    

def ride(here, there, system=boston):
    "Return a path on the subway system from here to there."
    ## your code here
    return shortest_path_search(here,lambda s:system[s],lambda s: there in s)
    

def longest_ride(system):
    """"Return the longest possible 'shortest path' 
    ride between any two stops in the system."""
    ## your code here
#     long_ride = []
#     for here in system:
#         for there in system:
#             ride = shortest_path_search(here,lambda s:system[s],lambda s: there in s)
#             long_ride = ride if len(ride)>len(long_ride) else long_ride
#     return long_ride
    stops = set(s for dic in boston.values() for s in dic)
    return max([ride(a,b) for a in stops for b in stops],key = len)
    

def shortest_path_search(start, successors, is_goal):
    """Find the shortest path from start state to a state
    such that is_goal(state) is true."""
    if is_goal(start):
        return [start]
    explored = set() # set of states we have visited
    frontier = [ [start] ] # ordered list of paths we have blazed
    while frontier:
        path = frontier.pop(0)
        s = path[-1]
        for (state, action) in successors(s).items():
            if state not in explored:
                explored.add(state)
                path2 = path + [action, state]
                if is_goal(state):
                    return path2
                else:
                    frontier.append(path2)
    return []

def path_states(path):
    "Return a list of states in this path."
    return path[0::2]
    
def path_actions(path):
    "Return a list of actions in this path."
    return path[1::2]
print(path_states(longest_ride(boston)))
def test_ride():
    assert ride('mit', 'government') == [
        'mit', 'red', 'charles', 'red', 'park', 'green', 'government']
    assert ride('mattapan', 'foresthills') == [
        'mattapan', 'red', 'umass', 'red', 'south', 'red', 'downtown',
        'orange', 'chinatown', 'orange', 'tufts', 'orange', 'backbay', 'orange', 'foresthills']
    assert ride('newton', 'alewife') == [
        'newton', 'green', 'kenmore', 'green', 'copley', 'green', 'park', 'red', 'charles', 'red',
        'mit', 'red', 'central', 'red', 'harvard', 'red', 'porter', 'red', 'davis', 'red', 'alewife']
    assert len(path_states(longest_ride(boston))) == 16
    assert (path_states(longest_ride(boston)) == ['wonderland', 'revere', 'suffolk', 'airport', 'maverick', 'aquarium', 'state', 'government', 'park', 'charles', 'mit', 'central', 'harvard', 'porter', 'davis', 'alewife']
            or path_states(longest_ride(boston)) == [
                'alewife', 'davis', 'porter', 'harvard', 'central', 'mit', 'charles', 
                'park', 'downtown', 'state', 'aquarium', 'maverick', 'airport', 'suffolk', 'revere', 'wonderland'])
    return 'test_ride passes'

print(test_ride())

['wonderland', 'revere', 'suffolk', 'airport', 'maverick', 'aquarium', 'state', 'government', 'park', 'charles', 'mit', 'central', 'harvard', 'porter', 'davis', 'alewife']
test_ride passes
